# FR-Train on clean synthetic data

## Import libraries

In [2]:
import sys, os
os.chdir('/Users/songtengyu/Documents/2023Fall/Information theory/fr-train/')

import numpy as np
import math

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch

import math
import matplotlib.pyplot as plt

from argparse import Namespace

from FRTrain_arch import Generator, DiscriminatorF, DiscriminatorR, weights_init_normal, test_model

import warnings
warnings.filterwarnings("ignore")

In [3]:
from scipy.stats import multivariate_normal

def multi_gaussian_pdf(x, mu, sigma):
    return multivariate_normal.pdf(x, mean=mu, cov=sigma)

In [5]:
def data_gen(ypos_p, zpos_coef, N_test = 7000):
    cov_y_0 = np.array([[10, 1], [1, 3]])
    cov_y_1 = np.array([[5, 1], [1, 5]])
    mu_y_0 = np.array([-2, -2])
    mu_y_1 = np.array([2, 2])

    X_test = np.zeros((N_test, 2))
    y_test = np.zeros((N_test))
    s1_test = np.zeros((N_test))

    for i in range(N_test):
        if np.random.rand() < 1 - ypos_p:
            X_test[i, :] = np.random.multivariate_normal([-2, -2], [[10, 1], [1, 3]])
            y_test[i] = 0
        else:
            X_test[i, :] = np.random.multivariate_normal([2, 2], [[5, 1], [1, 5]])
            y_test[i] = 1
        x_prime_1 = X_test[i, 0] * math.cos(math.pi/4) + X_test[i, 1] * math.sin(math.pi/4)
        x_prime_2 = - X_test[i, 0] * math.sin(math.pi/4) + X_test[i, 1] * math.cos(math.pi/4)
        p_z_1 = multi_gaussian_pdf([x_prime_1, x_prime_2], mu_y_1, cov_y_1) / \
                (multi_gaussian_pdf([x_prime_1, x_prime_2], mu_y_0, cov_y_0) + \
                multi_gaussian_pdf([x_prime_1, x_prime_2], mu_y_1, cov_y_1))
        if np.random.rand() < p_z_1 * zpos_coef:
            s1_test[i] = 1
        else:
            s1_test[i] = 0
    return X_test, y_test, s1_test

## Load and process data (using clean y train label)

In [6]:
# a namespace object which contains some of the hyperparameters
opt = Namespace(num_train=2000, num_val1=200, num_val2=500, num_test=1000)

In [7]:
num_train = opt.num_train
num_val1 = opt.num_val1
num_val2 = opt.num_val2
num_test = opt.num_test

X, y, s1 = data_gen(0.5, 0.5, 10000)

X = torch.FloatTensor(X)
y = torch.FloatTensor(y)
s1 = torch.FloatTensor(s1)

X_train = X[:num_train - num_val1]
y_train = y[:num_train - num_val1] # Clean label
s1_train = s1[:num_train - num_val1]


X_val = X[num_train: num_train + num_val1]
y_val = y[num_train: num_train + num_val1]
s1_val = s1[num_train: num_train + num_val1]

# Currently not used
# X_val2 = X[num_train + num_val1 : num_train + num_val1 + num_val2]
# y_val2 = y[num_train + num_val1 : num_train + num_val1 + num_val2]
# s1_val2 = s1[num_train + num_val1 : num_train + num_val1 + num_val2]

X_test = X[num_train + num_val1 + num_val2 : num_train + num_val1 + num_val2 + num_test]
y_test = y[num_train + num_val1 + num_val2 : num_train + num_val1 + num_val2 + num_test]
s1_test = s1[num_train + num_val1 + num_val2 : num_train + num_val1 + num_val2 + num_test]

XS_train = torch.cat([X_train, s1_train.reshape((s1_train.shape[0], 1))], dim=1)
XS_val = torch.cat([X_val, s1_val.reshape((s1_val.shape[0], 1))], dim=1)
XS_test = torch.cat([X_test, s1_test.reshape((s1_test.shape[0], 1))], dim=1)

In [8]:
print("--------------------- Number of Data -------------------------" )
print(
    "Train data : %d, Validation data : %d, Test data : %d "
    % (len(y_train), len(y_val), len(y_test))
)       
print("--------------------------------------------------------------")

--------------------- Number of Data -------------------------
Train data : 1800, Validation data : 200, Test data : 1000 
--------------------------------------------------------------


# Training with clean data

In [15]:
def train_model(train_tensors, val_tensors, test_tensors, train_opt, lambda_f, lambda_r, seed):
    """
      Trains FR-Train by using the classes in FRTrain_arch.py.
      
      Args:
        train_tensors: Training data.
        val_tensors: Clean validation data.
        test_tensors: Test data.
        train_opt: Options for the training. It currently contains size of validation set, 
                number of epochs, generator/discriminator update ratio, and learning rates.
        lambda_f: The tuning knob for L_2 (ref: FR-Train paper, Section 3.3).
        lambda_r: The tuning knob for L_3 (ref: FR-Train paper, Section 3.3).
        seed: An integer value for specifying torch random seed.
        
      Returns:
        Information about the tuning knobs (lambda_f, lambda_r),
        the test accuracy of the trained model, and disparate impact of the trained model.
    """
    
    XS_train = train_tensors.XS_train
    y_train = train_tensors.y_train
    s1_train = train_tensors.s1_train
    
    XS_val = val_tensors.XS_val
    y_val = val_tensors.y_val
    s1_val = val_tensors.s1_val
    
    XS_test = test_tensors.XS_test
    y_test = test_tensors.y_test
    s1_test = test_tensors.s1_test
    
    # Saves return values here
    test_result = []
    
    val = train_opt.val # Number of data points in validation set
    k = train_opt.k     # Updates ratio of generator and discriminator (1:k training).
    n_epochs = train_opt.n_epochs  # Number of training epoch
    
    # Changes the input validation data to an appropriate shape for the training
    XSY_val = torch.cat([XS_val, y_val.reshape((y_val.shape[0], 1))], dim=1)  

    # The loss values of each component will be saved in the following lists. 
    # We can draw epoch-loss graph by the following lists, if necessary.
    g_losses =[]
    d_f_losses = []
    d_r_losses = []
    clean_test_result = []

    bce_loss = torch.nn.BCELoss()

    # Initializes generator and discriminator
    generator = Generator()
    discriminator_F = DiscriminatorF()
    discriminator_R = DiscriminatorR()

    # Initializes weights
    torch.manual_seed(seed)
    generator.apply(weights_init_normal)
    discriminator_F.apply(weights_init_normal)
    discriminator_R.apply(weights_init_normal)

    optimizer_G = torch.optim.Adam(generator.parameters(), lr=train_opt.lr_g)
    optimizer_D_F = torch.optim.SGD(discriminator_F.parameters(), lr=train_opt.lr_f)
    optimizer_D_R = torch.optim.SGD(discriminator_R.parameters(), lr=train_opt.lr_r)

    XSY_val_data = XSY_val[:val]

    train_len = XS_train.shape[0]
    val_len = XSY_val.shape[0]

    # Ground truths using in Disriminator_R
    Tensor = torch.FloatTensor
    valid = Variable(Tensor(train_len, 1).fill_(1.0), requires_grad=False)
    generated = Variable(Tensor(train_len, 1).fill_(0.0), requires_grad=False)
    fake = Variable(Tensor(train_len, 1).fill_(0.0), requires_grad=False)
    clean = Variable(Tensor(val_len, 1).fill_(1.0), requires_grad=False)
    
    r_weight = torch.ones_like(y_train, requires_grad=False).float()
    r_ones = torch.ones_like(y_train, requires_grad=False).float()

    for epoch in range(n_epochs):

        # -------------------
        #  Forwards Generator
        # -------------------
        if epoch % k == 0 or epoch < 500:
            optimizer_G.zero_grad()

        gen_y = generator(XS_train)
        gen_data = torch.cat([XS_train, gen_y.detach().reshape((gen_y.shape[0], 1))], dim=1)
        
        # -----------------------------
        #  Trains Fairness Discriminator
        # -----------------------------

        optimizer_D_F.zero_grad()

        # Discriminator_F tries to distinguish the sensitive groups by using the output of the generator.
        d_f_loss= bce_loss(discriminator_F(gen_y.detach()).squeeze(), s1_train.squeeze())
        d_f_loss.backward()
        d_f_losses.append(d_f_loss)
        optimizer_D_F.step()
            

        # ---------------------------------
        #  Trains Robustness Discriminator
        # ---------------------------------
        optimizer_D_R.zero_grad()

        # Discriminator_R tries to distinguish whether the input is from the validation data or the generated data from generator.
        clean_loss = bce_loss(discriminator_R(XSY_val_data), clean)
        poison_loss = bce_loss(discriminator_R(gen_data.detach()), fake)
        d_r_loss = 0.5 * (clean_loss + poison_loss)

        d_r_loss.backward()
        d_r_losses.append(d_r_loss)
        optimizer_D_R.step()


        # ---------------------
        #  Updates Generator
        # ---------------------

        # Loss measures generator's ability to fool the discriminators
        if epoch < 500 :
            g_loss = bce_loss((F.tanh(gen_y).squeeze()+1)/2, (y_train.squeeze()+1)/2)
            g_loss.backward()
            g_losses.append(g_loss)
            optimizer_G.step()
    
        elif epoch % k == 0:
            r_decision = discriminator_R(gen_data)
            r_gen = bce_loss(r_decision, generated)
            
            # ------------------------------
            #  Re-weights using output of D_R
            # ------------------------------
            
            if epoch % 100 == 0:
                loss_ratio = (g_losses[-1]/d_r_losses[-1]).detach()
                a = 1/(1+torch.exp(-(loss_ratio-3)))
                b = 1-a
                r_weight_tmp = r_decision.detach().squeeze()
                r_weight = a * r_weight_tmp + b * r_ones

            f_cost = F.binary_cross_entropy(discriminator_F(gen_y).squeeze(), s1_train.squeeze(), reduction="none").squeeze()
            g_cost = F.binary_cross_entropy_with_logits(gen_y.squeeze(), (y_train.squeeze()+1)/2, reduction="none").squeeze()

            f_gen = torch.mean(f_cost*r_weight)
            g_loss = (1-lambda_f-lambda_r) * torch.mean(g_cost*r_weight) -  lambda_r * r_gen - lambda_f * f_gen

            g_loss.backward()
            optimizer_G.step()

        g_losses.append(g_loss)

        if epoch % 200 == 0:
            print(
                "[Lambda_f: %1f] [Epoch %d/%d] [D_F loss: %f] [D_R loss: %f] [G loss: %f]"
                % (lambda_f, epoch, n_epochs, d_f_losses[-1], d_r_losses[-1], g_losses[-1])
            )

#     torch.save(generator.state_dict(), './FR-Train_on_clean_synthetic.pth')
    tmp = test_model(generator, XS_test, y_test, s1_test)
    test_result.append([lambda_f, lambda_r, tmp[0].item(), tmp[1]])

    return test_result

In [16]:
train_result = []
train_tensors = Namespace(XS_train = XS_train, y_train = y_train, s1_train = s1_train)
val_tensors = Namespace(XS_val = XS_val, y_val = y_val, s1_val = s1_val) 
test_tensors = Namespace(XS_test = XS_test, y_test = y_test, s1_test = s1_test)

train_opt = Namespace(val=len(y_val), n_epochs=4000, k=3, lr_g=0.005, lr_f=0.01, lr_r=0.001)      
seed = 1 

lambda_f_set = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85] # Lambda value for the fairness discriminator of FR-Train.
lambda_r = 0.1 # Lambda value for the robustness discriminator of FR-Train.

for lambda_f in lambda_f_set:
    train_result.append(train_model(train_tensors, val_tensors, test_tensors, train_opt, lambda_f = lambda_f, lambda_r = lambda_r, seed=seed))

[Lambda_f: 0.200000] [Epoch 0/4000] [D_F loss: 0.689127] [D_R loss: 0.742629] [G loss: 0.640932]
[Lambda_f: 0.200000] [Epoch 200/4000] [D_F loss: 0.599112] [D_R loss: 0.718911] [G loss: 0.459559]
[Lambda_f: 0.200000] [Epoch 400/4000] [D_F loss: 0.563394] [D_R loss: 0.706668] [G loss: 0.453621]
[Lambda_f: 0.200000] [Epoch 600/4000] [D_F loss: 0.544496] [D_R loss: 0.693903] [G loss: 0.147935]
[Lambda_f: 0.200000] [Epoch 800/4000] [D_F loss: 0.530709] [D_R loss: 0.687867] [G loss: 0.143763]
[Lambda_f: 0.200000] [Epoch 1000/4000] [D_F loss: 0.522567] [D_R loss: 0.682616] [G loss: 0.142871]
[Lambda_f: 0.200000] [Epoch 1200/4000] [D_F loss: 0.519205] [D_R loss: 0.677272] [G loss: 0.142829]
[Lambda_f: 0.200000] [Epoch 1400/4000] [D_F loss: 0.519403] [D_R loss: 0.671837] [G loss: 0.143097]
[Lambda_f: 0.200000] [Epoch 1600/4000] [D_F loss: 0.521661] [D_R loss: 0.666355] [G loss: 0.143248]
[Lambda_f: 0.200000] [Epoch 1800/4000] [D_F loss: 0.524701] [D_R loss: 0.660833] [G loss: 0.143474]
[Lambda

In [17]:
print("--------------------------------------------------------------------------------")
print("------------------ Training Results of FR-Train on clean data ------------------" )
for i in range(len(train_result)):
    print(
        "[Lambda_f: %.2f] [Lambda_r: %.2f] Accuracy : %.3f, Disparate Impact : %.3f "
        % (train_result[i][0][0], train_result[i][0][1], train_result[i][0][2], train_result[i][0][3])
    )       
print("--------------------------------------------------------------------------------")

--------------------------------------------------------------------------------
------------------ Training Results of FR-Train on clean data ------------------
[Lambda_f: 0.20] [Lambda_r: 0.10] Accuracy : 0.627, Disparate Impact : 0.942 
[Lambda_f: 0.30] [Lambda_r: 0.10] Accuracy : 0.624, Disparate Impact : 0.984 
[Lambda_f: 0.40] [Lambda_r: 0.10] Accuracy : 0.624, Disparate Impact : 0.990 
[Lambda_f: 0.50] [Lambda_r: 0.10] Accuracy : 0.625, Disparate Impact : 0.998 
[Lambda_f: 0.60] [Lambda_r: 0.10] Accuracy : 0.625, Disparate Impact : 0.992 
[Lambda_f: 0.70] [Lambda_r: 0.10] Accuracy : 0.630, Disparate Impact : 0.986 
[Lambda_f: 0.80] [Lambda_r: 0.10] Accuracy : 0.641, Disparate Impact : 0.982 
[Lambda_f: 0.85] [Lambda_r: 0.10] Accuracy : 0.656, Disparate Impact : 0.991 
--------------------------------------------------------------------------------
